In [11]:
# Import required libraries
import pandas as pd
import numpy as np
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from matplotlib import pyplot
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.graphics.gofplots import qqplot
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error

In [12]:
data = pd.read_csv('energy_dataset.csv')

In [13]:
data.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 29 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   time                                         35064 non-null  object 
 1   generation biomass                           35045 non-null  float64
 2   generation fossil brown coal/lignite         35046 non-null  float64
 3   generation fossil coal-derived gas           35046 non-null  float64
 4   generation fossil gas                        35046 non-null  float64
 5   generation fossil hard coal                  35046 non-null  float64
 6   generation fossil oil                        35045 non-null  float64
 7   generation fossil oil shale                  35046 non-null  float64
 8   generation fossil peat                       35046 non-null  float64
 9   generation geothermal                        35046 non-null  float64
 10

In [15]:
data.describe()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
count,35045.000000,35046.000000,35046.0,35046.000000,35046.000000,35045.000000,35046.0,35046.0,35046.0,0.0,...,35045.000000,35046.0,35046.000000,35064.000000,0.0,35064.000000,35064.000000,35028.000000,35064.000000,35064.000000
mean,383.513540,448.059208,0.0,5622.737488,4256.065742,298.319789,0.0,0.0,0.0,NaN,...,269.452133,0.0,5464.479769,1439.066735,NaN,5471.216689,28712.129962,28696.939905,49.874341,57.884023
std,85.353943,354.568590,0.0,2201.830478,1961.601013,52.520673,0.0,0.0,0.0,NaN,...,50.195536,0.0,3213.691587,1677.703355,NaN,3176.312853,4594.100854,4574.987950,14.618900,14.204083
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,...,0.000000,0.0,0.000000,0.000000,NaN,237.000000,18105.000000,18041.000000,2.060000,9.330000
25%,333.000000,0.000000,0.0,4126.000000,2527.000000,263.000000,0.0,0.0,0.0,NaN,...,240.000000,0.0,2933.000000,69.000000,NaN,2979.000000,24793.750000,24807.750000,41.490000,49.347500
50%,367.000000,509.000000,0.0,4969.000000,4474.000000,300.000000,0.0,0.0,0.0,NaN,...,279.000000,0.0,4849.000000,576.000000,NaN,4855.000000,28906.000000,28901.000000,50.520000,58.020000
75%,433.000000,757.000000,0.0,6429.000000,5838.750000,330.000000,0.0,0.0,0.0,NaN,...,310.000000,0.0,7398.000000,2636.000000,NaN,7353.000000,32263.250000,32192.000000,60.530000,68.010000
max,592.000000,999.000000,0.0,20034.000000,8359.000000,449.000000,0.0,0.0,0.0,NaN,...,357.000000,0.0,17436.000000,5836.000000,NaN,17430.000000,41390.000000,41015.000000,101.990000,116.800000


In [16]:
data.isnull().sum()

time                                               0
generation biomass                                19
generation fossil brown coal/lignite              18
generation fossil coal-derived gas                18
generation fossil gas                             18
generation fossil hard coal                       18
generation fossil oil                             19
generation fossil oil shale                       18
generation fossil peat                            18
generation geothermal                             18
generation hydro pumped storage aggregated     35064
generation hydro pumped storage consumption       19
generation hydro run-of-river and poundage        19
generation hydro water reservoir                  18
generation marine                                 19
generation nuclear                                17
generation other                                  18
generation other renewable                        18
generation solar                              

In [17]:
# Missing Data

# Since it is a time-series, we will use linear interpolation to fill the missing data
df1 = data.interpolate(method='linear', axis=0).ffill().bfill()
df1.head(10)

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04
5,2015-01-01 05:00:00+01:00,410.0,178.0,0.0,4038.0,3590.0,156.0,0.0,0.0,0.0,...,188.0,0.0,4618.0,4.0,NaN,4617.0,21441.0,19905.0,35.72,53.63
6,2015-01-01 06:00:00+01:00,401.0,172.0,0.0,4040.0,3368.0,158.0,0.0,0.0,0.0,...,186.0,0.0,4397.0,3.0,NaN,4276.0,21285.0,20010.0,35.13,51.73
7,2015-01-01 07:00:00+01:00,408.0,172.0,0.0,4030.0,3208.0,160.0,0.0,0.0,0.0,...,189.0,0.0,3992.0,12.0,NaN,3994.0,21545.0,20377.0,36.22,51.43
8,2015-01-01 08:00:00+01:00,413.0,177.0,0.0,4052.0,3335.0,161.0,0.0,0.0,0.0,...,198.0,0.0,3629.0,39.0,NaN,3602.0,21443.0,20094.0,32.40,48.98
9,2015-01-01 09:00:00+01:00,419.0,177.0,0.0,4137.0,3437.0,163.0,0.0,0.0,0.0,...,198.0,0.0,3073.0,784.0,NaN,3212.0,21560.0,20637.0,36.60,54.20


In [18]:
df1.isnull().sum()

time                                               0
generation biomass                                 0
generation fossil brown coal/lignite               0
generation fossil coal-derived gas                 0
generation fossil gas                              0
generation fossil hard coal                        0
generation fossil oil                              0
generation fossil oil shale                        0
generation fossil peat                             0
generation geothermal                              0
generation hydro pumped storage aggregated     35064
generation hydro pumped storage consumption        0
generation hydro run-of-river and poundage         0
generation hydro water reservoir                   0
generation marine                                  0
generation nuclear                                 0
generation other                                   0
generation other renewable                         0
generation solar                              

In [19]:
# Drop empty columns
df2 = df1.drop(columns=['generation hydro pumped storage aggregated','forecast wind offshore eday ahead'])

df2.head(10)

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,49.0,196.0,0.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,50.0,195.0,0.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,50.0,196.0,0.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,50.0,191.0,0.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,42.0,189.0,0.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
5,2015-01-01 05:00:00+01:00,410.0,178.0,0.0,4038.0,3590.0,156.0,0.0,0.0,0.0,...,34.0,188.0,0.0,4618.0,4.0,4617.0,21441.0,19905.0,35.72,53.63
6,2015-01-01 06:00:00+01:00,401.0,172.0,0.0,4040.0,3368.0,158.0,0.0,0.0,0.0,...,34.0,186.0,0.0,4397.0,3.0,4276.0,21285.0,20010.0,35.13,51.73
7,2015-01-01 07:00:00+01:00,408.0,172.0,0.0,4030.0,3208.0,160.0,0.0,0.0,0.0,...,35.0,189.0,0.0,3992.0,12.0,3994.0,21545.0,20377.0,36.22,51.43
8,2015-01-01 08:00:00+01:00,413.0,177.0,0.0,4052.0,3335.0,161.0,0.0,0.0,0.0,...,54.0,198.0,0.0,3629.0,39.0,3602.0,21443.0,20094.0,32.40,48.98
9,2015-01-01 09:00:00+01:00,419.0,177.0,0.0,4137.0,3437.0,163.0,0.0,0.0,0.0,...,743.0,198.0,0.0,3073.0,784.0,3212.0,21560.0,20637.0,36.60,54.20


In [20]:
# Drop columns with only zeros
df2 = df2.loc[:, (df2 != 0).any(axis=0)]

df2.head(10)

,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,...,generation other renewable,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,...,73.0,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,...,71.0,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,...,73.0,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,...,75.0,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,...,74.0,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
5,2015-01-01 05:00:00+01:00,410.0,178.0,4038.0,3590.0,156.0,2109.0,952.0,743.0,7098.0,...,74.0,34.0,188.0,4618.0,4.0,4617.0,21441.0,19905.0,35.72,53.63
6,2015-01-01 06:00:00+01:00,401.0,172.0,4040.0,3368.0,158.0,2108.0,961.0,848.0,7098.0,...,74.0,34.0,186.0,4397.0,3.0,4276.0,21285.0,20010.0,35.13,51.73
7,2015-01-01 07:00:00+01:00,408.0,172.0,4030.0,3208.0,160.0,2031.0,983.0,1012.0,7099.0,...,72.0,35.0,189.0,3992.0,12.0,3994.0,21545.0,20377.0,36.22,51.43
8,2015-01-01 08:00:00+01:00,413.0,177.0,4052.0,3335.0,161.0,2119.0,1001.0,1015.0,7098.0,...,73.0,54.0,198.0,3629.0,39.0,3602.0,21443.0,20094.0,32.40,48.98
9,2015-01-01 09:00:00+01:00,419.0,177.0,4137.0,3437.0,163.0,2170.0,1041.0,1357.0,7097.0,...,74.0,743.0,198.0,3073.0,784.0,3212.0,21560.0,20637.0,36.60,54.20


In [21]:
df2.isnull().sum()

time                                           0
generation biomass                             0
generation fossil brown coal/lignite           0
generation fossil gas                          0
generation fossil hard coal                    0
generation fossil oil                          0
generation hydro pumped storage consumption    0
generation hydro run-of-river and poundage     0
generation hydro water reservoir               0
generation nuclear                             0
generation other                               0
generation other renewable                     0
generation solar                               0
generation waste                               0
generation wind onshore                        0
forecast solar day ahead                       0
forecast wind onshore day ahead                0
total load forecast                            0
total load actual                              0
price day ahead                                0
price actual        

In [22]:
# Drop forecase columns
df2 = df2.drop(columns=['forecast solar day ahead', 'forecast wind onshore day ahead', 'total load forecast', 'price day ahead'])

In [23]:
# Outlier Detection and Treatment 

# Detection using interquartile range
def iqr (s, k=1.5, thresholds=False):
    # calculate interquartile range
    q25, q75 = np.percentile(s, 25), np.percentile(s, 27)
    iqr = q75 - q25
    # calculate the outlier cutoff
    cut_off = iqr * k
    lower, upper = q25 - cut_off, q75 + cut_off
    if thresholds:
        return lower, upper
    else:
        # Identify outliers
        return [True if x < lower or x > upper else False for x in s]
    
# Outlier Treatment
# All outliers are replaced by nulls first.
# Linear interpolation is then used to fill the nulls

# Create df showing outliers
df3 = df2.drop(columns=['time'], axis=1)
iqr_df = df3.apply(iqr, k=1.5) 

iqr_df.head(10)

,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load actual,price actual
0,True,True,True,True,True,True,True,True,True,True,False,True,True,True,False,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True
4,True,True,False,True,True,True,True,True,True,True,False,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True
6,True,True,False,True,True,True,True,True,True,True,False,True,True,True,True,True
7,True,True,True,True,True,True,True,False,True,True,False,True,True,True,True,True
8,True,True,False,True,True,True,True,False,True,True,False,False,True,True,True,False
9,True,True,False,True,True,True,True,True,True,True,False,True,True,False,True,True


In [24]:
# Replace with nulls
for column in df3:
    df3[column] = np.where(iqr_df[column] == True, 'NaN', df3[column])

cols = df3.columns
df3[cols] = df3[cols].apply(pd.to_numeric, errors='coerce') 

# Fill the nulls
df4 = df3.interpolate(method='linear', axis=0).bfill().ffill()
df2['time'] = pd.to_datetime(df2['time'], utc=True)
df4 = pd.concat([df2['time'], df4], axis=1)
df4.head(10)

,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load actual,price actual
0,2014-12-31 23:00:00+00:00,334.0,0.0,4130.0,2559.0,257.0,0.0,677.0,1012.00,6000.0,54.0,73.0,54.000000,251.0,3073.0,25385.000000,48.980
1,2015-01-01 00:00:00+00:00,334.0,0.0,4130.0,2559.0,257.0,0.0,677.0,1012.00,6000.0,54.0,73.0,54.000000,251.0,3073.0,25333.307692,48.980
2,2015-01-01 01:00:00+00:00,334.0,0.0,4130.0,2559.0,257.0,0.0,677.0,1012.00,6000.0,54.0,73.0,54.000000,251.0,3073.0,25281.615385,48.980
3,2015-01-01 02:00:00+00:00,334.0,0.0,4130.0,2559.0,257.0,0.0,677.0,1012.00,6000.0,54.0,75.0,54.000000,251.0,3073.0,25229.923077,48.980
4,2015-01-01 03:00:00+00:00,334.0,0.0,4130.0,2559.0,257.0,0.0,677.0,1012.00,6000.0,54.0,74.0,54.000000,251.0,3073.0,25178.230769,48.980
5,2015-01-01 04:00:00+00:00,334.0,0.0,4085.0,2559.0,257.0,0.0,677.0,1012.00,6000.0,54.0,74.0,54.000000,251.0,3073.0,25126.538462,48.980
6,2015-01-01 05:00:00+00:00,334.0,0.0,4040.0,2559.0,257.0,0.0,677.0,1012.00,6000.0,54.0,74.0,54.000000,251.0,3073.0,25074.846154,48.980
7,2015-01-01 06:00:00+00:00,334.0,0.0,4046.0,2559.0,257.0,0.0,677.0,1012.00,6000.0,54.0,72.0,54.000000,251.0,3073.0,25023.153846,48.980
8,2015-01-01 07:00:00+00:00,334.0,0.0,4052.0,2559.0,257.0,0.0,677.0,1015.00,6000.0,54.0,73.0,54.000000,251.0,3073.0,24971.461538,48.980
9,2015-01-01 08:00:00+00:00,334.0,0.0,4137.0,2559.0,257.0,0.0,677.0,1062.25,6000.0,54.0,74.0,57.833333,251.0,3073.0,24919.769231,49.004


In [25]:
# Drop columns with only zeros
df5 = df4.loc[:, (df4 != 0).any(axis=0)]

# Drop colunms with the same values
df5 = df5.drop(df5.std()[(df5.std() == 0)].index, axis=1)

df5.head(10)

,time,generation biomass,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load actual,price actual
0,2014-12-31 23:00:00+00:00,334.0,4130.0,2559.0,257.0,677.0,1012.00,6000.0,54.0,73.0,54.000000,251.0,3073.0,25385.000000,48.980
1,2015-01-01 00:00:00+00:00,334.0,4130.0,2559.0,257.0,677.0,1012.00,6000.0,54.0,73.0,54.000000,251.0,3073.0,25333.307692,48.980
2,2015-01-01 01:00:00+00:00,334.0,4130.0,2559.0,257.0,677.0,1012.00,6000.0,54.0,73.0,54.000000,251.0,3073.0,25281.615385,48.980
3,2015-01-01 02:00:00+00:00,334.0,4130.0,2559.0,257.0,677.0,1012.00,6000.0,54.0,75.0,54.000000,251.0,3073.0,25229.923077,48.980
4,2015-01-01 03:00:00+00:00,334.0,4130.0,2559.0,257.0,677.0,1012.00,6000.0,54.0,74.0,54.000000,251.0,3073.0,25178.230769,48.980
5,2015-01-01 04:00:00+00:00,334.0,4085.0,2559.0,257.0,677.0,1012.00,6000.0,54.0,74.0,54.000000,251.0,3073.0,25126.538462,48.980
6,2015-01-01 05:00:00+00:00,334.0,4040.0,2559.0,257.0,677.0,1012.00,6000.0,54.0,74.0,54.000000,251.0,3073.0,25074.846154,48.980
7,2015-01-01 06:00:00+00:00,334.0,4046.0,2559.0,257.0,677.0,1012.00,6000.0,54.0,72.0,54.000000,251.0,3073.0,25023.153846,48.980
8,2015-01-01 07:00:00+00:00,334.0,4052.0,2559.0,257.0,677.0,1015.00,6000.0,54.0,73.0,54.000000,251.0,3073.0,24971.461538,48.980
9,2015-01-01 08:00:00+00:00,334.0,4137.0,2559.0,257.0,677.0,1062.25,6000.0,54.0,74.0,57.833333,251.0,3073.0,24919.769231,49.004


In [26]:
df5.isnull().sum()

time                                          0
generation biomass                            0
generation fossil gas                         0
generation fossil hard coal                   0
generation fossil oil                         0
generation hydro run-of-river and poundage    0
generation hydro water reservoir              0
generation nuclear                            0
generation other                              0
generation other renewable                    0
generation solar                              0
generation waste                              0
generation wind onshore                       0
total load actual                             0
price actual                                  0
dtype: int64

In [27]:
# Data Partitioning

# 70-30 split
X = df5.values
train_size = int(len(X) * 0.7)
train, test = X[0:train_size], X[train_size:len(X)]

print('Observations: %d' % (len(X)))
print('Training Observations: %d' % (len(train)))
print('Testing Observations: %d' % (len(test)))

Observations: 35064
Training Observations: 24544
Testing Observations: 10520


In [29]:
# Naive Forecast

# Reframe the time series data as a supervised learning dataset

# Create a lag feature
var = pd.DataFrame(series.values)
dataframe = pd.concat([var.shift(1), var], axis=1)
dataframe.columns = ['t', 't+1']
print(dataframe.head(5))

#train
#train[0,2:5]

array([4130.0, 2559.0, 257.0], dtype=object)